# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Aluno**:

**RA**: 


---
### Análise exploratória

Nesta seção, deve ser feita a leitura da base de dados e todas as análises necessárias para interpretar e analisar os dados, tais como:
* Significado de cada atributo
* Medidas descritivas
* Gráficos

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/raw_data/news_data_17_May_2020.csv
data/raw_data/news_data_19_Jul_2020.csv
data/raw_data/news_data_12_Dec_2019.csv
data/raw_data/news_data_15_Mar_2020.csv
data/raw_data/news_data_16_Apr_2020.csv
data/raw_data/news_data_14_Feb_2020.csv
data/raw_data/news_data_08_Aug_2019.csv
data/raw_data/news_data_01_Jan_2019.csv
data/raw_data/news_data_09_Sep_2019.csv
data/raw_data/news_data_23_Nov_2020.csv
data/raw_data/news_data_24_Dec_2020.csv
data/raw_data/news_data_03_Mar_2019.csv
data/raw_data/news_data_21_Sep_2020.csv
data/raw_data/news_data_02_Feb_2019.csv
data/raw_data/news_data_04_Apr_2019.csv
data/raw_data/news_data_07_Jul_2019.csv
data/raw_data/news_data_22_Oct_2020.csv
data/raw_data/news_data_20_Aug_2020.csv
data/raw_data/news_data_18_Jun_2020.csv
data/raw_data/train.csv
data/raw_data/news_data_06_Jun_2019.csv
data/raw_data/news_data_05_May_2019.csv
data/raw_data/news_data_10_Oct_2019.csv
data/raw_data/sample_submission.csv
data/raw_data/test.csv
data/raw_data/news_data_11_Nov_2019.c

Primeira tentativa: o conjunto de treino foi somente os 3 primeiros meses


In [2]:
# first_sample = pd.read_csv("data/raw_data/news_data_01_Jan_2019.csv") #
# sec_sample = pd.read_csv("data/raw_data/news_data_02_Feb_2019.csv")   # Juntos formam o X
# third_sample = pd.read_csv("data/raw_data/news_data_03_Mar_2019.csv") #

train_csv = pd.read_csv("data/raw_data/train.csv")                    # É o Y (com os labels)

test_csv = pd.read_csv("data/raw_data/test.csv")                      # Teste

sample_sub = pd.read_csv("data/raw_data/sample_submission.csv")       # Submissão

alldata = pd.read_csv("data/pre_processed_data/all_news.csv")    

In [ ]:
# diretorio = '/kaggle/input/ufscar-am-2023-2-projeto-final'

#arquivos_csv = [arquivo for arquivo in os.listdir(diretorio) if arquivo.startswith('new') and arquivo.endswith('.csv')]

#dados_concatenados = pd.DataFrame()

#for arquivo in arquivos_csv:
 #   print(arquivo)
  #  caminho_arquivo = os.path.join(diretorio, arquivo)
   # dados = pd.read_csv(caminho_arquivo)
    #dados_concatenados = pd.concat([dados_concatenados, dados], ignore_index=True)

#dados_concatenados.to_csv(f'/kaggle/working/all_news.csv', index=False)

In [4]:
test = test_csv.merge(alldata, on = "id")
test

,id,title,content,date
0,id-3378296,WATCH LIVE: Kayleigh McEnany holds White House...,White House Press Secretary Kayleigh McEnany i...,2020-09-24
1,id-3138916,Phil Collins funny moment…,Post was not sent - check your email addresses...,2020-08-07
2,id-3304259,House panel investigating postmaster general o...,Washington — The House Oversight and Reform Co...,2020-09-08
3,id-3381436,Trump won’t commit to a ‘peaceful transfer of ...,"“ Well , we ’ re going to have to see what hap...",2020-09-24
4,id-3594175,AP PHOTOS: Worldly concerns bleed into the spi...,A figure of a saint consumed by flames after p...,2020-11-01
...,...,...,...,...
167703,id-3606259,Catholic Foster Care Agency Should Be Able to ...,Catholic Foster Care Agency Should Be Able to ...,2020-11-05
167704,id-3028526,Egypt desperate to revive coronavirus-hit tour...,"A wonder of the world , the pyramids at Giza h...",2020-07-15
167705,id-3458862,Coronavirus UK news – Test and trace CHAOS as ...,BRITAIN 'S calamitous test and trace system ha...,2020-10-08
167706,id-3729241,Recount Confirms Biden’s Wisconsin Win,“ President Trump blasted judges ’ rejections ...,2020-11-29


In [5]:
joinned_train = train_csv.merge(alldata, on = "id")
joinned_train

,id,label,title,content,date
0,id-1000000,NaN,"Germany and Russia: natural partners, so why n...",This [ post ] ( https : //www.fort-russ.com/20...,2019-01-01
1,id-1000001,1.0,Netflix Pulls Muslim Comedy Show After Saudi C...,"The in Hollywood continues to eat itself , as ...",2019-01-01
2,id-1000002,NaN,New Study Reports that Grass Really IS Greener...,While the adage that the grass is always green...,2019-01-01
3,id-1000003,0.0,NaN,"Daniel Mihailescu , AFP | The government build...",2019-01-01
4,id-1000004,0.0,Nasri completes West Ham move as transfer wind...,Former France midfielder Samir Nasri has compl...,2019-01-01
...,...,...,...,...,...
1947879,id-2947879,0.0,Chicagoans mourn victims of gun violence after...,Treja Kelley had been hesitant to come forward...,2020-06-30
1947880,id-2947880,1.0,HITTING AIR POCKETS: Masses Choking – DEATH BY...,"There are no BOOM AND BUST cycles anymore , at...",1940-06-30
1947881,id-2947881,0.0,Coronavirus in Illinois updates: Here’s what’s...,Illinois health officials Monday reported 738 ...,2020-06-30
1947882,id-2947882,NaN,FEMA Camp Has Begun Homeless First,"In August , the Colombia City Council unanimou...",2020-06-30


In [6]:
joinned_train.dropna(inplace = True)
joinned_train.drop(columns = [ "id", "date" ], inplace = True)

In [7]:
test.drop(columns = [ "id", "date" ], inplace = True)

In [10]:
joinned_train['title_content'] = joinned_train['title'] + " " + joinned_train['content']
test['title_content'] = test['title'] + " " + test['content']

: 

In [ ]:
# train = pd.concat([first_sample, sec_sample, third_sample])

In [ ]:
#joinned_train = pd.read_csv("/kaggle/input/alldata-joinned-train/alldata_joinned_train.csv")

In [ ]:
# colocar tudo em minusculo
# tirar acentuação
# tirar stopwords

import nltk
from nltk.corpus import stopwords
import re
import string

PUNCT_TO_REMOVE = string.punctuation
STOPWORDS = set(stopwords.words('english'))

nltk.download('stopwords')

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def lower(text):
    return text.lower()

def preprocess(text):
    if pd.isnull(text):  # Verifica se é NaN
        return ""

    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = lower(text)

    return text

---
### Pré-processamento

Nesta seção, as funções da etapa de pré-processamento dos dados devem ser implementadas e aplicadas (se necessário).

In [ ]:
#joinned_train["title"] = joinned_train["title"].apply(lambda text: preprocess(text))
#test["title"] = test["title"].apply(lambda text: preprocess(text))

In [ ]:
joinned_train["content"] = joinned_train["content"].apply(lambda text: preprocess(text))
test["content"] = test["content"].apply(lambda text: preprocess(text))

In [ ]:
joinned_train.to_csv("preprocessed_joined_train.csv")

In [ ]:
test.to_csv("preprocessed_test.csv")

---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais padrões e testando diferentes modelos.

In [ ]:
X = joinned_train["content"]
y_train = joinned_train["label"]
X_test = test["content"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer(max_features = 8000)
x_train = vectorization.fit_transform(X)
X_test_vec = vectorization.transform(X_test)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(objective='binary:logistic')
xgb.fit(x_train, y_train)
pred = xgb.predict_proba(X_test_vec)

In [ ]:
sample_sub["label"] = pred[:, 1] 
sample_sub.set_index("id").to_csv("default-xgb-alldata.csv")

---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos através de tabelas e gráficos, comparados e profundamente analisados.